# Transfer Learning using ResNet50


## Libraries

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.models import load_model
from keras.datasets import cifar10
from keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

## ResNet50

In [2]:
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

94765736/94765736 [==============================] - 0s 0us/step


In [3]:
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

## Adding New Layers

In [4]:
model = models.Sequential()
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10, activation='softmax'))

## Dataset

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


## Preprocessing

### Normalization

In [6]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

## To binary (for use with categorical_crossentropy)

In [7]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## Training the Model

In [8]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=15, batch_size=20, validation_data=(x_test, y_test))

Epoch 1/15
2500/2500 [==============================] - 695s 260ms/step - loss: 0.3409 - acc: 0.1736 - val_loss: 7.6155 - val_acc: 0.1307
Epoch 2/15
2500/2500 [==============================] - 648s 259ms/step - loss: 0.2821 - acc: 0.2522 - val_loss: 0.2672 - val_acc: 0.3135
Epoch 3/15
2500/2500 [==============================] - 649s 259ms/step - loss: 0.2629 - acc: 0.3197 - val_loss: 0.2493 - val_acc: 0.3574
Epoch 4/15
2500/2500 [==============================] - 650s 260ms/step - loss: 0.2427 - acc: 0.4126 - val_loss: 0.4392 - val_acc: 0.4860
Epoch 5/15
2500/2500 [==============================] - 652s 261ms/step - loss: 0.2198 - acc: 0.5004 - val_loss: 0.2970 - val_acc: 0.4173
Epoch 6/15
2500/2500 [==============================] - 654s 261ms/step - loss: 0.2011 - acc: 0.5632 - val_loss: 0.1756 - val_acc: 0.6062
Epoch 7/15
2500/2500 [==============================] - 655s 262ms/step - loss: 0.1857 - acc: 0.6112 - val_loss: 0.2789 - val_acc: 0.6252
Epoch 8/15
2500/2500 [============

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', test_accuracy)

In [ ]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels

In [ ]:
cm = confusion_matrix(y_test, y_pred_classes)
print(cm)

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
model.save('ResNet50modelCifar-10')

# Training ResNet50 from Scratch

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
# Create ResNet50 model without pre-trained weights
base_model = ResNet50(weights=None, include_top=False, input_shape=(32, 32, 3))

# Add classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Optional dense layer for more capacity
predictions = Dense(10, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train,
          epochs=50,
          batch_size=64,
          validation_data=(x_test, y_test))


Epoch 1/50
506/782 [==================>...........] - ETA: 13s - loss: 1.9541 - accuracy: 0.3275